In [ ]:
from firedrake import *
import icepack.plot
import numpy as np

In [ ]:
mesh = UnitSquareMesh(10, 10)
R = 100
mesh.coordinates.dat.data[:] *= R

x, y = SpatialCoordinate(mesh)
Q = FunctionSpace(mesh, "CG", 1)
V = VectorFunctionSpace(mesh, "CG", 1)
phi = firedrake.TestFunction(Q)
psi = firedrake.TrialFunction(Q)
M = phi * psi * dx
n = FacetNormal(mesh)

In [ ]:
fig, axes = icepack.plot.subplots()
icepack.plot.triplot(mesh, axes=axes)
axes.set_title('Mesh');
axes.legend();

In [ ]:
# Set the velocity
x_velocity = Constant(0)
y_velocity = Constant(-2)
u = interpolate(as_vector([x_velocity,y_velocity]),V)

# Set the initial age field and age function
a0 = interpolate(R-y,Q)
a = Function(Q)
a.assign(a0)
da = Function(Q)

# Set the time step and source (which eaquals the time step)
dt = 1
source = Constant(dt)

In [ ]:
u_n = max_value(0, inner(u, n))
f = a * u_n
flux_faces = f * phi * dS
flux_cells = -a * div(u * phi) * dx
flux_out = a * max_value(0, inner(u, n)) * phi * ds
flux_in = Constant(0) * min_value(0, inner(u, n)) * phi * ds
flux = flux_cells + flux_out + flux_in
J = -dt * flux

In [ ]:
# Loop through time steps
parameters = {
        'solver_parameters': {
            'ksp_type': 'preonly',
            'pc_type': 'bjacobi',
            'sub_pc_type': 'ilu'
        }
    }
ntsteps = 20
for t in range(ntsteps):
    problem1 = LinearVariationalProblem(M, J, da)
    solver1 = LinearVariationalSolver(problem1, **parameters)
    solver1.solve()
    a.assign(a + da + (source))

In [ ]:
fig, axes = icepack.plot.subplots()
contours = icepack.plot.tripcolor(interpolate(a0,Q), vmin=-80, vmax=80, cmap='RdBu_r', axes=axes)
fig.colorbar(contours)
axes.set_title('Initial Age');

In [ ]:
fig, axes = icepack.plot.subplots()
contours = icepack.plot.tripcolor(interpolate(a,Q), vmin=-80, vmax=80, cmap='RdBu_r', axes=axes)
fig.colorbar(contours)
axes.set_title('Final Age');

In [ ]:
fig, axes = icepack.plot.subplots()
contours = icepack.plot.tripcolor(interpolate(a-a0,Q), vmin=-ntsteps-10, vmax=ntsteps+10, cmap='RdBu_r', axes=axes)
fig.colorbar(contours)
axes.set_title('Age Difference (Initial - Final)');

In [ ]:
# Implement a check with the known solution.

In [ ]:
# Old stuff - gets a nonlinear diverge and wouldn't run anyways without setting up more of the variables

In [ ]:
# Use a theta family time discretization of the age evolution equation
# Crank-Nicolson (theta = 1/2)
# Backward Euler (theta = 1)
theta = Constant(0.)
at = s - inner(u,grad(a))
at_prior = s - inner(u,grad(a_prior))
da = a-a_prior
J_main = (da/s - theta*at - (1-theta)*at_prior) * phi * dx

# Add in some boundary conditions
# Daniel prefers using a max_value statement because it's more obvious than the algebra in the next two lines
un_pos = 0.5*(dot(u, n) + abs(dot(u, n))) 
un_neg = 0.5*(dot(u, n) - abs(dot(u, n)))
q_inflow = Constant(0)

J_inflow = q_inflow * un_neg * phi * ds
J_outflow = a * un_pos * phi * ds

# Total action functional is:
J_left = J_main + J_outflow + J_inflow
J_right = 0

In [ ]:
# Loop through time steps
import numpy as np
ntsteps = 1 # There's a convergence error if I go above one time step
for t in range(ntsteps):
    solve(J_left == J_right, a)#, bcs=[bc])
    a_prior.assign(a)
    a = Function(Q)